# Model #1 : student generation and parameters learning

In [1]:
import sys

sys.path.append("/Users/olivier/PycharmProjects/bayesian-kst/")

import numpy as np
from kgraph.expert_layer.domain_graph import DomainGraph
from kgraph.expert_layer.knowledge_components import KnowledgeComponent, ProceduralKnowledgeComponent
from kgraph.expert_layer.links import LinkModel, LinkFromParents, LinkFromChildren
from kgraph.resources_layer.exercise_family import ExerciseFamily
from kgraph.resources_layer.exercise import Exercise
from kgraph.learner_layer.evaluation import Evaluation
from kgraph.learner_layer.learner_pool import LearnerPool
from kgraph.learner_layer.learner import Learner, LearnerGraph
from kgraph.helpers.truthtable import truthtable
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.dynamicBN as gdyn

## Student generation

In [2]:
# we define the KCs
KC_A = ProceduralKnowledgeComponent(1, "A")
KC_B = ProceduralKnowledgeComponent(2, "B")
KC_C = ProceduralKnowledgeComponent(3, "C")

P_C_knowing_A_and_B = np.array([[1., 0.],
                                [.85, .15],
                                [.9, .1],
                                [.4, .6]])


# we define the link between the two that gonna lead to the link model
link_a_to_b = LinkFromParents(KC_B, [KC_A], [.4, .6])
link_b_to_a = LinkFromChildren(KC_A, [KC_B], [.1, .9])
link_model = LinkModel([LinkFromParents(KC_C, [KC_A, KC_B], P_C_knowing_A_and_B.flatten().tolist())])
# we create the associated domain graph
domain_graph = DomainGraph([KC_A, KC_B, KC_C], link_model)

In [3]:


# default params
params = {"slip": .1, "guess":.25}

# we define the exercises that corresponds to KC_A
ex_A_1 = Exercise(1, "qcm", ex_content="", params=params)
ex_A_2 = Exercise(2, "qcm", ex_content="", params=params)
ex_A_3 = Exercise(3, "qcm", ex_content="", params=params)
ex_A_4 = Exercise(4, "qcm", ex_content="", params=params)
ex_A_5 = Exercise(5, "qcm", ex_content="", params=params)

ex_fam_A = ExerciseFamily(1, "ex_fam_A", KC_A, [ex_A_1, ex_A_2, ex_A_3, ex_A_4, ex_A_5])

# we define the exercises that corresponds to KC_A
ex_B_1 = Exercise(6, "qcm", ex_content="", params=params)
ex_B_2 = Exercise(7, "qcm", ex_content="", params=params)
ex_B_3 = Exercise(8, "qcm", ex_content="", params=params)
ex_B_4 = Exercise(9, "qcm", ex_content="", params=params)
ex_B_5 = Exercise(10, "qcm", ex_content="", params=params)

ex_fam_B = ExerciseFamily(1, "ex_fam_B", KC_B, [ex_B_1, ex_B_2, ex_B_3, ex_B_4, ex_B_5])


# we define the exercises that corresponds to KC_C
ex_C_1 = Exercise(11, "qcm", ex_content="", params=params)
ex_C_2 = Exercise(12, "qcm", ex_content="", params=params)
ex_C_3 = Exercise(13, "qcm", ex_content="", params=params)
ex_C_4 = Exercise(14, "qcm", ex_content="", params=params)
ex_C_5 = Exercise(15, "qcm", ex_content="", params=params)

ex_fam_B = ExerciseFamily(1, "ex_fam_B", KC_C, [ex_C_1, ex_C_2, ex_C_3, ex_C_4, ex_C_5])


In [4]:
learner_pool = LearnerPool(domain_graph)
learner_pool.setup_random_learners(10)

In [5]:
learner_pool.print_default_learner_graph()
for learner in learner_pool.learners:
    print(learner.learner_graph)

LearnerGraph of Learner 0
P(1) = 0.2 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})
P(2) = 0.2 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})
P(3) = 0.2 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})

LearnerGraph of Learner 1
P(1) = 0.6762301327293652 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})
P(2) = 0.21364441528947864 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})
P(3) = 0.15023515252419042 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})

LearnerGraph of Learner 2
P(1) = 0.016812306301324242 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})
P(2) = 0.07784216582820702 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})
P(3) = 0.01381560272163528 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})

LearnerGraph of Learner 3
P(1) = 0.5252442015826245 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})
P(2) = 0.4789587568297141 (params: {'learn': 0.1, 'delta': -0.9, 'gamma': 2.2})
P(3) = 0.2124178421204299 (params: {'learn': 0.1, 'd

In [6]:
print([learner.id for learner in learner_pool.learners])
sim_evaluations = learner_pool.simulate_evaluations_from_learners(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [7]:
knowledge_components = domain_graph.get_knowledge_components()

import pandas as pd
n_dbn_iter = max([len(sim_evaluations[i][:]) for i in range(len(sim_evaluations))])
col_names = [[[f'{kc.name}_{k}', f'eval_{kc.name}_{k}'] for kc in knowledge_components] for k in range(5 * n_dbn_iter)]
col_names = [element for li in col_names for element in li]
col_names = [element for li in col_names for element in li]

print(len(col_names))

data = pd.DataFrame([["?" for _ in range(5*n_dbn_iter*len(knowledge_components)*2)] for _ in range(len(sim_evaluations))],
                   columns = col_names)


for i in range(len(sim_evaluations)):  # pour chaque élève i
    for j in range(len(sim_evaluations[i])):  # pour chaque eval k de l'élève i
        evaluated_kc = sim_evaluations[i][j].get_kc()
        for k in range(len(sim_evaluations[i][j].answers.keys())):
            step_idx = 5*j + k
            key = list(sim_evaluations[i][j].answers)[k]
            data.iloc[i][f"eval_{evaluated_kc.name}_{step_idx}"] = int(sim_evaluations[i][j].answers[key]['success'])
            

60


In [8]:
print(data)
data.to_csv("missing_values_data.csv")

  A_0 eval_A_0 B_0 eval_B_0 C_0 eval_C_0 A_1 eval_A_1 B_1 eval_B_1  ... B_8  \
0   ?        ?   ?        ?   ?        ?   ?        ?   ?        ?  ...   ?   
1   ?        ?   ?        ?   ?        ?   ?        ?   ?        ?  ...   ?   
2   ?        ?   ?        0   ?        ?   ?        ?   ?        1  ...   ?   
3   ?        ?   ?        1   ?        ?   ?        ?   ?        0  ...   ?   
4   ?        ?   ?        0   ?        ?   ?        ?   ?        1  ...   ?   
5   ?        ?   ?        ?   ?        ?   ?        ?   ?        ?  ...   ?   
6   ?        ?   ?        1   ?        ?   ?        ?   ?        1  ...   ?   
7   ?        ?   ?        1   ?        ?   ?        ?   ?        0  ...   ?   
8   ?        ?   ?        ?   ?        ?   ?        ?   ?        ?  ...   ?   
9   ?        ?   ?        ?   ?        0   ?        ?   ?        ?  ...   ?   

  eval_B_8 C_8 eval_C_8 A_9 eval_A_9 B_9 eval_B_9 C_9 eval_C_9  
0        ?   ?        ?   ?        ?   ?        ?   ?        ?  


## 1) Réalisation de l'exercice associé à KC A

### Classic BKT
**Nodes** :
- for every KC for every step of the evaluation on an exercise family
- for every exercise result in the exercise family associated to the evaluation

**Links**:
- between evaluated KC nodes between successive temporalities 
- between evaluated KC and its parents/children for a same temporality
- between evaluated KC and exercise result node

In [9]:
# we initialize the network
bkt_net = gum.BayesNet('BKT')

# we setup every node
for i in range(n_dbn_iter*5):
    for kc in domain_graph.get_knowledge_components():
        bkt_net.add(gum.LabelizedVariable(f"{kc.name}_{i}", '', 2))
        bkt_net.add(gum.LabelizedVariable(f"eval_{kc.name}_{i}", '', 2))
        bkt_net.addArc(*(f"{kc.name}_{i}", f"eval_{kc.name}_{i}"))
        if i!=0:
            bkt_net.addArc(*(f"{kc.name}_{i-1}", f"{kc.name}_{i}"))
    for kc in domain_graph.get_knowledge_components():
        kc_parents = domain_graph.link_model.get_parents(kc)
        for parent in kc_parents:
            bkt_net.addArc(*(f"{parent.name}_{i}", f"{kc.name}_{i}"))
        

In [10]:
# we initialize the network
fast_bkt_net_str = ""
# we setup every node
for i in range(n_dbn_iter*5):
    for kc in domain_graph.get_knowledge_components():
        fast_bkt_net_str += f"{kc.name}_{i}" + "->" + f"eval_{kc.name}_{i}" + ";"
        if i!=0:
            fast_bkt_net_str += f"{kc.name}_{i-1}" + "->" + f"{kc.name}_{i}" + ";"
    for kc in domain_graph.get_knowledge_components():
        kc_parents = domain_graph.link_model.get_parents(kc)
        for parent in kc_parents:
            fast_bkt_net_str += f"{parent.name}_{i}" + "->" + f"{kc.name}_{i}" + ";"

fast_bkt_net = gum.fastBN(fast_bkt_net_str[:-1])

In [11]:
learner = gum.BNLearner("missing_values_data.csv", fast_bkt_net)
print(f"Missing values in missing_values_data.csv : {learner.hasMissingValues()}")



Missing values in missing_values_data.csv : True


In [12]:
fast_bkt_net

(gum::BayesNet<double>@0x12761d800) BN{nodes: 60, arcs: 77, domainSize: 10^18.0618, dim: 348}

In [13]:
learner.setVerbosity(True)
learner.useEM(1e-3)
bn=learner.learnParameters(fast_bkt_net.dag())
print(f"# iterations : {learner.nbrIterations()}")
gnb.sideBySide(gnb.getInference(fast_bkt_net),gnb.getInference(bn))


# iterations : 4
